In [3]:
#necessary
import polars as pl#similar to pandas, but with better performance when dealing with large datasets.
import pandas as pd#read csv,parquet
import numpy as np#for scientific computation of matrices
#model
from sklearn.linear_model import Ridge
import os#Libraries that interact with the operating system
import warnings#avoid some negligible errors
#The filterwarnings () method is used to set warning filters, which can control the output method and level of warning information.
warnings.filterwarnings('ignore')

import random#provides some functions for generating random numbers
#set random seed,make sure model can be recurrented.
def seed_everything(seed):
    np.random.seed(seed)#numpy random seed
    random.seed(seed)#python built-in random seed
seed_everything(seed=2025)

In [ ]:
def custom_metric(y_true,y_pred,weight):
    weighted_r2=1-(np.sum(weight*(y_true-y_pred)**2)/np.sum(weight*y_true**2))
    return weighted_r2
    
print("< read parquet >")
datas=[]
for i in range(7,10):
    train=pl.read_parquet(f"./train.parquet/partition_id={i}/part-0.parquet")
    train=train.to_pandas()
    datas.append(train)
train=pd.concat(datas)
print(f"train.shape:{train.shape}")

print("< get X,y >")
cols=[f'feature_0{i}' if i<10 else f'feature_{i}' for i in range(79)]
X=train[cols].fillna(3).values
y=train['responder_6'].values

print("< train test split >")
split=1300000#around 10%
weights=train['weight'].values
train_X,train_y,test_X,test_y,train_weight,test_weight=X[:-split],y[:-split],X[-split:],y[-split:],weights[:-split],weights[-split:]
print(f"train_X.shape:{train_X.shape},test_X.shape:{test_X.shape}")

print("< fit and predict >")
model=Ridge()
model.fit(train_X,train_y)
train_pred=model.predict(train_X)
test_pred=model.predict(test_X)
print(f"train weighted_r2:{custom_metric(train_y,train_pred,weight=train_weight)}")
print(f"test weighted_r2:{custom_metric(test_y,test_pred,weight=test_weight)}")